In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [4]:
url = 'https://ru.wikipedia.org/wiki/Список_станций_Московского_метрополитена'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
table = soup.find_all('table')[2]

In [5]:
titles = []
for i in table.find_all('th'):
  title = i.text.strip()
  titles.append(title)
data = pd.DataFrame(columns = titles)
for j in table.find_all('tr')[1:]:
  row_data = j.find_all('td')
  row = [i.text.strip() for i in row_data]
  length = len(data)
  data.loc[length] = row
coords = data[['Название станцииПрежние названия', 'Координаты']]
coord = coords.set_index('Название станцииПрежние названия')['Координаты'].to_dict()
coord = {key: value.split('″\xa0с.\xa0ш. ') for key, value in coord.items()}
coord = {key: [value[0], value[1].split('″\xa0в.\xa0д.HЯ')[0]] for key, value in coord.items()}
coord = {key.split('«')[0]: [int(value[0].split('°')[0])+int(value[0].split('°')[1].split('′')[0])/60+int(value[0].split('°')[1].split('′')[1])/3600, int(value[1].split('°')[0])+int(value[1].split('°')[1].split('′')[0])/60+int(value[1].split('°')[1].split('′')[1])/3600] for key, value in coord.items()}
centr_stations = {
    key: value for key, value in coord.items()
    if 55.72916665 < value[0] < 55.7811112 and 37.5774 < value[1] < 37.659445
}
centr_stations

{'Комсомольская': [55.77472222222222, 37.655],
 'Красные Ворота': [55.76888888888889, 37.64861111111111],
 'Чистые пруды': [55.76583333333333, 37.638888888888886],
 'Лубянка': [55.75972222222222, 37.62722222222222],
 'Охотный Ряд': [55.757777777777775, 37.61666666666667],
 'Библиотека имени Ленина': [55.75111111111111, 37.61],
 'Кропоткинская': [55.74527777777778, 37.603611111111114],
 'Парк культуры': [55.73583333333333, 37.59138888888889],
 'Белорусская': [55.77638888888889, 37.58444444444444],
 'Маяковская': [55.769999999999996, 37.59583333333334],
 'Тверская': [55.764722222222225, 37.60638888888889],
 'Театральная': [55.757777777777775, 37.61888888888889],
 'Новокузнецкая': [55.74138888888889, 37.629444444444445],
 'Павелецкая': [55.73166666666667, 37.63777777777778],
 'Смоленская': [55.74888888888889, 37.5825],
 'Арбатская': [55.751666666666665, 37.600833333333334],
 'Площадь Революции': [55.75666666666667, 37.62166666666667],
 'Курская': [55.75694444444444, 37.659444444444446],
 

In [121]:
import plotly.express as px
df_metro = pd.DataFrame(centr_stations).T.reset_index()
df_metro.columns = ['Название', 'Широта', 'Долгота']
fig = px.scatter_mapbox(
    df_metro,
    lat = 'Широта',
    lon = 'Долгота',
    hover_name = 'Название',
    zoom = 12,
    height = 600
)
fig.update_layout(
    mapbox_style = "carto-positron",
    margin = {"r": 0, "t": 0, "l": 0, "b": 0},
)
fig.show()

In [7]:
overall = []
all_items = []
api_key = "Ваш АПИ"

query = "Фастфуд"

min_lon = 37.5774
min_lat = 55.72916665
max_lon = 37.659445
max_lat = 55.7811112
for i in range(4):
  for j in range(4):
    left_lon = min_lon + j*(max_lon-min_lon)/4
    right_lon = min_lon + (j+1)*(max_lon-min_lon)/4
    down_lat = min_lat + i*(max_lat-min_lat)/4
    up_lat = min_lat + (i+1)*(max_lat-min_lat)/4
    page_size = 10
    page = 1
    while page<=5:
        url = f"https://catalog.api.2gis.com/3.0/items?q={query}&point1={left_lon},{up_lat}&point2={right_lon},{down_lat}&fields=items.point,items.name,items.address,items.reviews&page_size={page_size}&page={page}&key={api_key}"
        response = requests.get(url)
        data = response.json()
        if data['meta']['code'] == 200:
            for item in data["result"]["items"]:
                all_items.append(item)
            page += 1
        else:
            page += 5
            pass
for item in all_items:
  name = item["name"]
  coordinates = item["point"]
  if 'general_rating' in item['reviews'].keys():
    rating = item['reviews']['general_rating']
    a = {'Название': name, 'Координаты': coordinates, 'Рейтинг': rating}
    overall.append(a)
  else:
    pass
overall


[{'Название': 'Бургер Кинг, сеть ресторанов быстрого питания',
  'Координаты': {'lat': 55.736297, 'lon': 37.590943},
  'Рейтинг': 4},
 {'Название': 'Rostic`s, ресторан быстрого обслуживания',
  'Координаты': {'lat': 55.736297, 'lon': 37.590943},
  'Рейтинг': 2.6},
 {'Название': 'СССР, чебуречная',
  'Координаты': {'lat': 55.736283, 'lon': 37.590985},
  'Рейтинг': 3.1},
 {'Название': 'Кулинарная лавка братьев Караваевых',
  'Координаты': {'lat': 55.735667, 'lon': 37.587116},
  'Рейтинг': 3.8},
 {'Название': 'Pattaya, кафе быстрого обслуживания',
  'Координаты': {'lat': 55.734792, 'lon': 37.588482},
  'Рейтинг': 5},
 {'Название': 'ХХХ, кафе грузинской кухни',
  'Координаты': {'lat': 55.734792, 'lon': 37.588482},
  'Рейтинг': 4.5},
 {'Название': 'Папа Джонс, сеть пиццерий',
  'Координаты': {'lat': 55.733407, 'lon': 37.592568},
  'Рейтинг': 3.7},
 {'Название': 'Wave California poke, кафе',
  'Координаты': {'lat': 55.734792, 'lon': 37.588482},
  'Рейтинг': 4.9},
 {'Название': 'Crosta, пицца

In [8]:
graph = pd.DataFrame(overall)
graph['lat'] = graph['Координаты'].apply(lambda x: x['lat'])
graph['lon'] = graph['Координаты'].apply(lambda x: x['lon'])
fig = px.scatter_mapbox(
    graph,
    lat='lat',
    lon='lon',
    hover_name='Название',
    color='Рейтинг',
    zoom=12,
    height=600
)
fig.update_layout(
    mapbox_style="carto-positron",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)
fig.show()
len(graph['lat'])


600

In [95]:
#Скачиваем файл объявлений с ЦИАНа, подходящих под необходимые параметры
arenda = pd.read_excel('/content/1.xlsx',sheet_name='ЦИАН - Аренда коммерческой')

In [11]:
all_arenda = {}
api_key = "Ваш АПИ"
for i in range(len(arenda['Адрес'])):
  url = f"https://geocode-maps.yandex.ru/1.x/?apikey={api_key}&geocode={arenda['Адрес'][i]}&format=json"
  response = requests.get(url)
  data = response.json()
  feature = data["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]
  coord = feature["Point"]["pos"].split()
  if 55.72916665 < float(coord[1]) < 55.7811112 and 37.5774 < float(coord[0]) < 37.659445:
    all_arenda[arenda['Адрес'][i]] = [float(coord[0]), float(coord[1]), int(arenda['Цена'][i].split()[0])]
all_arenda

{'Москва, улица Земляной Вал, 25': [37.658185, 55.759734, 710000],
 'Москва, улица Зацепский Вал, 4С2': [37.639869, 55.733031, 450000],
 'Москва, улица Грузинский Вал, 28/45': [37.582906, 55.774907, 525000],
 'Москва, Валовая улица, 8/18': [37.632125, 55.730887, 825000],
 'Москва, улица Солянка, 2/6': [37.636823, 55.754005, 1450000],
 'Москва, Новослободская улица, 19С1': [37.599193, 55.780654, 394999],
 'Москва, улица Земляной Вал, 32': [37.657125, 55.757009, 70000],
 'Москва, улица Грузинский Вал, 26С1': [37.581613, 55.774472, 100000],
 'Москва, Пятницкая улица, 16С1': [37.628155, 55.742534, 195000],
 'Москва, проспект Мира, 44': [37.633868, 55.780967, 260000],
 'Москва, улица Большая Дмитровка, 32С1': [37.610844, 55.7656, 437500],
 'Москва, Богословский переулок, 16/6С1': [37.598106, 55.76369, 1150000],
 'Москва, Большой Афанасьевский переулок, 27С2': [37.596273, 55.748554, 4000],
 'Москва, Ленинградский проспект, 1': [37.580409, 55.777753, 200000],
 'Москва, улица Маросейка, 9/2С1'

In [12]:
# В одном объявлении указана цена за м^2, поэтому правим в ручную
all_arenda['Москва, Большой Афанасьевский переулок, 27С2'][2] = 4000*70
all_arenda

{'Москва, улица Земляной Вал, 25': [37.658185, 55.759734, 710000],
 'Москва, улица Зацепский Вал, 4С2': [37.639869, 55.733031, 450000],
 'Москва, улица Грузинский Вал, 28/45': [37.582906, 55.774907, 525000],
 'Москва, Валовая улица, 8/18': [37.632125, 55.730887, 825000],
 'Москва, улица Солянка, 2/6': [37.636823, 55.754005, 1450000],
 'Москва, Новослободская улица, 19С1': [37.599193, 55.780654, 394999],
 'Москва, улица Земляной Вал, 32': [37.657125, 55.757009, 70000],
 'Москва, улица Грузинский Вал, 26С1': [37.581613, 55.774472, 100000],
 'Москва, Пятницкая улица, 16С1': [37.628155, 55.742534, 195000],
 'Москва, проспект Мира, 44': [37.633868, 55.780967, 260000],
 'Москва, улица Большая Дмитровка, 32С1': [37.610844, 55.7656, 437500],
 'Москва, Богословский переулок, 16/6С1': [37.598106, 55.76369, 1150000],
 'Москва, Большой Афанасьевский переулок, 27С2': [37.596273,
  55.748554,
  280000],
 'Москва, Ленинградский проспект, 1': [37.580409, 55.777753, 200000],
 'Москва, улица Маросейка, 

In [129]:
df = pd.DataFrame.from_dict(all_arenda, orient='index', columns=['Долгота', 'Широта', 'Цена аренды'])
df.reset_index(inplace=True)
df.rename(columns={'index': 'Адрес'}, inplace=True)

fig = px.scatter_mapbox(
    df,
    lat='Широта',
    lon='Долгота',
    hover_name='Адрес',
    hover_data={'Цена аренды': True, 'Широта': False, 'Долгота': False},
    color='Цена аренды',
    size='Цена аренды',
    zoom=11.8,
    height=600
)

fig.update_layout(
    mapbox_style="carto-positron",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    title={'text': 'Цена аренды в Москве', 'x': 0.5, 'xanchor': 'center'}
)

fig.show()

Пусть в отсутствии конкурентов и с нормальным спросом (без учёта бонуса от метро) наша перепечная принесёт выручки за вычетом затрат на ингридиенты -- 90000. Наша перепечная очевидно прайстейкер, т.к. рынок фастфуда приближен к совершенной конкуренции, поэтому пусть представим вышеупомянутую величину как $90000D$, где $D$ -- величина спроса. Положим, что заведение фастфуда с оценкой 5.0, находящееся в одной точке с нами забирает половину спроса на нашу продукцию. Пусть доля спроса, забираемого конкурентом прямо пропорционально зависит от оценки конкурента и чем дальше он от нас, тем меньшую долю нашего спроса он забирает. Если расстояние до конкурента более километра (12 минут пешком), то логично предположить, что эффектом на наш спрос можно пренебречь; пусть зависимость от расстояния линейна.
  
Так долю спроса, забираемого конкурентами, можно представить как $(Rating_1(1-R_1)+...Rating_i(1-R_i))/5(i+1)$, где $R$ -- расстояние в км до конкурентов, находящихся на расстоянии меньше километра от нас (их i штук), а $Rating$ -- их рейтинг.

Чем ближе мы к метро, тем больше людей через нас проходят, поэтому к нашей функции выручки можно добавить бонус за метро поблизости: пусть, если расстояние до станции метро составляет меньше километра, то спрос домножается на коэффициент 2 - M, где M -- расстояние до этой станции метро.

Итого функцию прибыли можно записать как

$90000(2-M_i)...(2-M_j)(1-(Rating1(1−R1)+...Ratingi(1−Ri))/5(i+1)) - Rent$

где $Rent$ -- арендная плата.

In [92]:
#Длина одного градуса долготы на широте 55.7°: 62.80 км
#Длина одного градуса широты: 111.32 км
arenda_info = {}

for adres in all_arenda:
    near_places = []
    near_stations = []
    for metro in range(len(df_metro)):
      lon_dif = abs(all_arenda[adres][0]-df_metro['Долгота'][metro])*62.8
      lat_dif = abs(all_arenda[adres][1]-df_metro['Широта'][metro])*111.32
      M = (lon_dif**2+lat_dif**2)**0.5
      if M < 1:
        near_stations.append([df_metro['Название'][metro], round(float(M),3)])
    for place in range(len(graph)):
      lon_dif = abs(all_arenda[adres][0]-graph['lon'][place])*62.8
      lat_dif = abs(all_arenda[adres][1]-graph['lat'][place])*111.32
      R = (lon_dif**2+lat_dif**2)**0.5
      if R < 1:
        near_places.append([graph['Название'][place], round(float(R),3), float(graph['Рейтинг'][place])])
    arenda_info[adres] = [near_stations, near_places, all_arenda[adres][2]]


In [120]:
max_money = 0
dash_info = {}
for adres in arenda_info:
  D = 1
  part_stolen = 0
  neighbors = 0
  for stations in arenda_info[adres][0]:
    D = D * (2 - stations[1])
  for places in arenda_info[adres][1]:
    neighbors += 1
    part_stolen += (1 - places[1])*places[2]
  part = 1 - part_stolen/(5*(neighbors+1))
  D = D * part
  money = 90000 * D - arenda_info[adres][2]
  dash_info[adres] = [arenda_info[adres][0], arenda_info[adres][1], arenda_info[adres][2], round(money,2)]
  if money - max_money > 0:
    max_money = round(money,2)
    answer = adres
print(answer, max_money)
dash_info





Москва, улица Большая Дмитровка, 32С1 199886.01


{'Москва, улица Земляной Вал, 25': [[['Курская', 0.32], ['Чкаловская', 0.377]],
  [['Вкусно — и точка, ресторан быстрого питания', 0.127, 2.8],
   ['Пока никто не видит, пивной бар', 0.813, 3.8],
   ['Mates Bistro', 0.935, 4.3],
   ['Бургер Кинг, сеть ресторанов быстрого питания', 0.179, 3.9],
   ['Джаганнат, вегетарианское кафе и магазин', 0.121, 3.6],
   ['Pizza Maestrello, кафе-пиццерия', 0.782, 4.7],
   ['Carl`s Jr., кафе фастфуда', 0.187, 4.4],
   ['Кулинарная лавка братьев Караваевых', 0.807, 4.1],
   ['Теремок, блинная', 0.191, 4.2],
   ['Rostic`s, ресторан быстрого обслуживания', 0.175, 2.4],
   ['PhoBo, кафе вьетнамской кухни', 0.216, 4.9],
   ['Donutto, пончиковая', 0.973, 2.1],
   ['Блины у камина, блинная лавка', 0.565, 4.5],
   ['#Farш, бургерная', 0.17, 2.7],
   ['Soul in the bowl, кафе быстрого обслуживания', 0.212, 4.9],
   ['Николай, бар-буфет', 0.531, 4.6],
   ['Ван Гог, гастробар', 0.105, 3.8],
   ['Шаверма по-московски', 0.823, 3.9],
   ['ВкусВилл, кафе', 0.491, 4.3

In [113]:
!pip install dash
import dash
from dash import dcc, html, dash_table


In [119]:


app = dash.Dash(__name__)

# Установка белого фона для всего дэшборда
app.layout = html.Div(style={'backgroundColor': 'white'}, children=[
    dcc.Tabs(id="tabs", value='Москва, улица Земляной Вал, 25', children=[
        dcc.Tab(label=key, value=key) for key in dash_info.keys()
    ]),
    html.Div(id='tabs-content')
])

@app.callback(
    dash.dependencies.Output('tabs-content', 'children'),
    [dash.dependencies.Input('tabs', 'value')]
)
def render_content(tab):
    # Проверяем, есть ли данные для выбранной вкладки
    info = dash_info[tab]


    metro_stations = info[0]
    establishments = info[1]
    rent = info[2]
    profit = info[3]

    # Создаем DataFrame для таблицы метро
    metro_df = pd.DataFrame(metro_stations, columns=['Станция метро', 'Расстояние (км)'])

    # Создаем таблицу для метро
    metro_table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in metro_df.columns],
        data=metro_df.to_dict('records'),
        style_table={'margin': '20px', 'width': '50%'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left', 'padding': '10px'}
    )

    # Создаем DataFrame для заведений
    establishments_df = pd.DataFrame(establishments, columns=['Заведение', 'Расстояние (км)', 'Оценка'])

    # Создаем график для заведений
    establishments_fig = {
        'data': [
            {'x': establishments_df['Расстояние (км)'],
             'y': establishments_df['Оценка'],
             'text': establishments_df['Заведение'],
             'mode': 'markers',
             'marker': {'size': 10}}
        ],
        'layout': {
            'title': 'Заведения поблизости',
            'xaxis': {'title': 'Расстояние (км)'},
            'yaxis': {'title': 'Оценка'},
            'hovermode': 'closest'
        }
    }

    return html.Div([
        html.H3(tab, style={'textAlign': 'center', 'color': 'black'}),
        html.H4('Станции метро поблизости', style={'color': 'black'}),
        metro_table,
        html.H4('Заведения поблизости', style={'color': 'black'}),
        dcc.Graph(figure=establishments_fig),
        html.H4(f'Месячная аренда: {rent} руб.', style={'color': 'black'}),
        html.H4(f'Ожидаемая прибыль: {profit} руб.', style={'color': 'black'})
    ])

if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>